In [1]:
from create_overview import ResultAggregator
from copy import deepcopy
import re
from ast import literal_eval
from statistics import mean
import numpy as np
from collections import defaultdict
import pandas as pd

In [2]:
wandb_api_key = "3b7e4cee8259b9c6522aa3adef3833e6c109f000"

In [3]:
#ra = ResultAggregator(wandb_api_key=wandb_api_key, verbose_logging=False)
ra = ResultAggregator(path_to_csv_export="current_wandb_results_unprocessed.csv", verbose_logging=False, score='weighted-f1')
#ra.get_info()

/Users/vetonmatoshi/Documents/Git/LEXTREME/utils/create_overview.py:78: DtypeWarning: Columns (43,74,79,101,110,117) have mixed types. Specify dtype option on import or set low_memory=False.
  results = pd.read_csv(Path(path_to_csv_export))


In [4]:
ra.create_report()

/Users/vetonmatoshi/Documents/Git/LEXTREME/utils/create_overview.py:279: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_pivot['mean_over_seeds']=df_pivot.mean(axis=1)
/Users/vetonmatoshi/Documents/Git/LEXTREME/utils/create_overview.py:279: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_pivot['mean_over_seeds']=df_pivot.mean(axis=1)
/Users/vetonmatoshi/Documents/Git/LEXTREME/utils/create_overview.py:279: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_pivot['mean_over_seeds']=df_pi

In [5]:
ra.macro_f1_overview

seed,datasets,finetuning_task,_name_or_path,language,1,2,3,mean_over_seeds
0,brazilian_court_decisions,brazilian_court_decisions_judgment,distilbert-base-multilingual-cased,pt,0.280780,0.286266,0.290629,0.285892
1,brazilian_court_decisions,brazilian_court_decisions_judgment,microsoft/Multilingual-MiniLM-L12-H384,pt,0.434852,0.576685,0.469231,0.493589
2,brazilian_court_decisions,brazilian_court_decisions_judgment,microsoft/mdeberta-v3-base,pt,0.655753,0.693536,0.624596,0.657961
3,brazilian_court_decisions,brazilian_court_decisions_judgment,neuralmind/bert-base-portuguese-cased,pt,0.595290,0.715128,0.624258,0.644892
4,brazilian_court_decisions,brazilian_court_decisions_judgment,xlm-roberta-base,pt,0.599061,0.614389,0.670780,0.628077
...,...,...,...,...,...,...,...,...
176,swiss_judgment_prediction,swiss_judgment_prediction,distilbert-base-multilingual-cased,all,0.662036,0.679172,0.659664,0.666957
177,swiss_judgment_prediction,swiss_judgment_prediction,microsoft/Multilingual-MiniLM-L12-H384,all,0.671795,0.667385,0.690082,0.676421
178,swiss_judgment_prediction,swiss_judgment_prediction,microsoft/mdeberta-v3-base,all,0.681537,0.696978,0.691897,0.690137
179,swiss_judgment_prediction,swiss_judgment_prediction,xlm-roberta-base,all,0.462327,0.459987,0.460698,0.461004


In [13]:
df = ra.results[(ra.results.seed.isin([1,2,3]))][['finetuning_task', '_name_or_path', 'language', 'seed', 'predict/_'+ra.score]]
df = df.pivot_table(values='predict/_'+ra.score, index=['finetuning_task', '_name_or_path', 'language'], columns='seed')
datasets = [ra.meta_infos['config_to_dataset'][i[0]] for i in df.index.tolist()]
df['datasets']=datasets
df.reset_index(inplace=True)
df = df[['datasets','finetuning_task', '_name_or_path', 'language',1, 2, 3]]
df

seed,datasets,finetuning_task,_name_or_path,language,1,2,3
0,brazilian_court_decisions,brazilian_court_decisions_judgment,distilbert-base-multilingual-cased,pt,0.443552,0.447331,0.449455
1,brazilian_court_decisions,brazilian_court_decisions_judgment,microsoft/Multilingual-MiniLM-L12-H384,pt,0.404121,0.619449,0.489076
2,brazilian_court_decisions,brazilian_court_decisions_judgment,microsoft/mdeberta-v3-base,pt,0.708249,0.735545,0.674590
3,brazilian_court_decisions,brazilian_court_decisions_judgment,neuralmind/bert-base-portuguese-cased,pt,0.616507,0.750373,0.669722
4,brazilian_court_decisions,brazilian_court_decisions_judgment,xlm-roberta-base,pt,0.641621,0.645067,0.720030
...,...,...,...,...,...,...,...
176,swiss_judgment_prediction,swiss_judgment_prediction,distilbert-base-multilingual-cased,all,0.759086,0.783557,0.752321
177,swiss_judgment_prediction,swiss_judgment_prediction,microsoft/Multilingual-MiniLM-L12-H384,all,0.774618,0.764423,0.795725
178,swiss_judgment_prediction,swiss_judgment_prediction,microsoft/mdeberta-v3-base,all,0.786136,0.801131,0.802062
179,swiss_judgment_prediction,swiss_judgment_prediction,xlm-roberta-base,all,0.724685,0.724676,0.724043


In [36]:
ra.meta_infos.keys()

dict_keys(['task_code_mapping', 'task_language_mapping', 'config_to_dataset', 'dataset_to_config', 'language_models', 'model_language_lookup_table'])

In [ ]:
ra.meta_infos['model_language_lookup_table']

In [ ]:
ra.results[ra.results.finetuning_task=='legal_code_subject']

In [ ]:
for lm, language in ra.meta_infos['model_language_lookup_table'].items():
    if language =="pt":
        print(lm)

In [ ]:
for model, language in ra.meta_infos['model_language_lookup_table'].items():
    #print(language)
    if language in ra.meta_infos['task_language_mapping']['lener_br']:
        print(model)

In [6]:
task_constraint = ["brazilian_court_decisions_judgment",  "brazilian_court_decisions_unanimity", "german_argument_mining", "covid19_emergency_event", "online_terms_of_service_clause_topics"]
print(task_constraint)

['brazilian_court_decisions_judgment', 'brazilian_court_decisions_unanimity', 'german_argument_mining', 'covid19_emergency_event', 'online_terms_of_service_clause_topics']


In [7]:
task = "covid19_emergency_event"
relevant_languages = ra.meta_infos['task_language_mapping'][task]
relevant_columns = [l+'_predict/_micro-f1' for l in relevant_languages]

In [8]:
ra.results[ra.results.finetuning_task=="covid19_emergency_event"][['_name_or_path','predict/_macro-f1']+relevant_columns]

,_name_or_path,predict/_macro-f1,hu_predict/_micro-f1,it_predict/_micro-f1,pl_predict/_micro-f1,nl_predict/_micro-f1,en_predict/_micro-f1,fr_predict/_micro-f1,nb_predict/_micro-f1
203,roberta-base,0.391684,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205,pdelobelle/robbert-v2-dutch-base,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207,dkleczek/bert-base-polish-uncased-v1,0.133929,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209,dbmdz/bert-base-italian-cased,0.319973,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210,dbmdz/bert-base-french-europeana-cased,0.374948,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212,camembert-base,0.139061,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214,SZTAKI-HLT/hubert-base-cc,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215,Musixmatch/umberto-commoncrawl-cased-v1,0.167917,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216,GroNLP/bert-base-dutch-cased,0.050000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
218,roberta-base,0.361575,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
ra.reset_list_of_available_languages()
print('Results without all available languages:')
print('------------------------')
ra.get_language_aggregated_score(task_constraint=task_constraint)
#ra.get_dataset_aggregated_score()
ra.get_dataset_aggregated_score(average_over_language=False, task_constraint=task_constraint)

lm = ["distilbert-base-multilingual-cased", "microsoft/Multilingual-MiniLM-L12-H384", "xlm-roberta-base", "microsoft/mdeberta-v3-base", "xlm-roberta-large"]

for x in lm:
    print(x)
    print('\t','language aggregated: ',ra.language_aggregated_score.at[x,'aggregated_score'])
    print('\t','dataset aggregated: ', ra.dataset_aggregated_score.at[x,'aggregated_score'])
    print('-----------------------------------')

print('\n##########################\n')


Results without all available languages:
------------------------
distilbert-base-multilingual-cased
	 language aggregated:  0.48861360074916105
	 dataset aggregated:  0.6520721799124827
-----------------------------------
microsoft/Multilingual-MiniLM-L12-H384
	 language aggregated:  0.22324876125613233
	 dataset aggregated:  0.4856849496050887
-----------------------------------
xlm-roberta-base
	 language aggregated:  0.5987761897123878
	 dataset aggregated:  0.7110509969718489
-----------------------------------
microsoft/mdeberta-v3-base
	 language aggregated:  0.5784956892004716
	 dataset aggregated:  0.730167281326757
-----------------------------------
xlm-roberta-large
	 language aggregated:  0.649072768606473
	 dataset aggregated:  0.7497476793676701
-----------------------------------

##########################



In [13]:
ra.reset_list_of_available_languages()
print('Results without all available languages:')
print('------------------------')
ra.get_language_aggregated_score(task_constraint=task_constraint)
ra.get_dataset_aggregated_score(task_constraint=task_constraint)
#ra.get_dataset_aggregated_score(average_over_language=False)

lm = ["distilbert-base-multilingual-cased", "microsoft/Multilingual-MiniLM-L12-H384", "xlm-roberta-base", "microsoft/mdeberta-v3-base", "xlm-roberta-large"]

for x in lm:
    print(x)
    print('\t','language aggregated: ',ra.language_aggregated_score.at[x,'aggregated_score'])
    print('\t','dataset aggregated: ', ra.dataset_aggregated_score.at[x,'aggregated_score'])
    print('-----------------------------------')

print('\n##########################\n')


Results without all available languages:
------------------------
distilbert-base-multilingual-cased
	 language aggregated:  0.5148200080622769
	 dataset aggregated:  0.6377788570517194
-----------------------------------
microsoft/Multilingual-MiniLM-L12-H384
	 language aggregated:  0.23907356140796845
	 dataset aggregated:  0.48107960308183684
-----------------------------------
xlm-roberta-base
	 language aggregated:  0.624932135886598
	 dataset aggregated:  0.7041335602007563
-----------------------------------
microsoft/mdeberta-v3-base
	 language aggregated:  0.5991812334623619
	 dataset aggregated:  0.7094621215238357
-----------------------------------
xlm-roberta-large
	 language aggregated:  0.6405061665257538
	 dataset aggregated:  0.7202359684702259
-----------------------------------

##########################



In [14]:
ra.reset_list_of_available_languages()
print('Results without all available languages:')
print('------------------------')
ra.get_language_aggregated_score(task_constraint=task_constraint)
ra.get_dataset_aggregated_score(task_constraint=task_constraint)

lm = ["distilbert-base-multilingual-cased", "microsoft/Multilingual-MiniLM-L12-H384", "xlm-roberta-base", "microsoft/mdeberta-v3-base", "xlm-roberta-large"]

for x in lm:
    print(x)
    print('\t','language aggregated: ',ra.language_aggregated_score.at[x,'aggregated_score'])
    print('\t','dataset aggregated: ', ra.dataset_aggregated_score.at[x,'aggregated_score'])
    print('-----------------------------------')

print('\n##########################\n')

print('Results without Hungarian:')
print('------------------------')
ra.remove_languages(['hu'])
ra.get_language_aggregated_score(task_constraint=task_constraint)
ra.get_dataset_aggregated_score(task_constraint=task_constraint)


for x in lm:
    print(x)
    print('\t','language aggregated: ',ra.language_aggregated_score.at[x,'aggregated_score'])
    print('\t','dataset aggregated: ', ra.dataset_aggregated_score.at[x,'aggregated_score'])
    print('-----------------------------------')

print('\n##########################\n')


ra.reset_list_of_available_languages()

print('Results without Dutch:')
print('------------------------')
ra.remove_languages(['nl'])
ra.get_language_aggregated_score(task_constraint=task_constraint)
ra.get_dataset_aggregated_score(task_constraint=task_constraint)


for x in lm:
    print(x)
    print('\t','language aggregated: ',ra.language_aggregated_score.at[x,'aggregated_score'])
    print('\t','dataset aggregated: ', ra.dataset_aggregated_score.at[x,'aggregated_score'])
    print('-----------------------------------')

print('\n##########################\n')

ra.reset_list_of_available_languages()

print('Results without Dutch and Hungarian:')
print('------------------------')
ra.remove_languages(['nl','hu'])
ra.get_language_aggregated_score(task_constraint=task_constraint)
ra.get_dataset_aggregated_score(task_constraint=task_constraint)


for x in lm:
    print(x)
    print('\t','language aggregated: ',ra.language_aggregated_score.at[x,'aggregated_score'])
    print('\t','dataset aggregated: ', ra.dataset_aggregated_score.at[x,'aggregated_score'])
    print('-----------------------------------')

print('\n##########################\n')


Results without all available languages:
------------------------
distilbert-base-multilingual-cased
	 language aggregated:  0.5148200080622769
	 dataset aggregated:  0.6377788570517194
-----------------------------------
microsoft/Multilingual-MiniLM-L12-H384
	 language aggregated:  0.23907356140796845
	 dataset aggregated:  0.48107960308183684
-----------------------------------
xlm-roberta-base
	 language aggregated:  0.624932135886598
	 dataset aggregated:  0.7041335602007563
-----------------------------------
microsoft/mdeberta-v3-base
	 language aggregated:  0.5991812334623619
	 dataset aggregated:  0.7094621215238357
-----------------------------------
xlm-roberta-large
	 language aggregated:  0.6405061665257538
	 dataset aggregated:  0.7202359684702259
-----------------------------------

##########################

Results without Hungarian:
------------------------
distilbert-base-multilingual-cased
	 language aggregated:  0.5791725090700615
	 dataset aggregated:  0.65795654

In [ ]:
from datasets import load_dataset

dataset = load_dataset("joelito/lextreme", task)
train_set = pd.DataFrame(dataset['train'])
print(train_set.language.unique())

In [ ]:
train_set[train_set.language=="en"].shape

In [ ]:
train_set

In [ ]:
language = "nb"
train_set[train_set.language==language]

In [ ]:
[l for l in train_set[train_set.language==language].label.tolist() if l!=[]]

In [ ]:
import json as js

with open('../meta_infos.json', 'r') as f:
    data = js.load(f)
data

In [ ]:
# Creating a dictionary to look up the language for each language model 
model_language_lookup_table = dict()
for k, v in data["language_models"].items():
    if type(v) == dict:
        for language, info in v.items():
            if language == "multilingual":
                language = "all"
            for size, models in info.items():
                for m in models:
                    model_language_lookup_table[m] = language


In [ ]:
model_language_lookup_table